# Simple Case Using Cramer Rao Bound

Set up a spring-mass-damper system as a simple example to try this concept. Start by defining the system dynamics. 

In [126]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from filterpy.kalman import KalmanFilter
from scipy.linalg import expm
from scipy.signal import cont2discrete

# Spring-mass-damper parameters
m = 1.0  # Mass (kg)
c = 0.5  # Damping coefficient (Ns/m)
k = 2.0  # Spring constant (N/m)

# Continuous-time state-space matrices
A_c = np.array([[0, 1],
                [-k/m, -c/m]])
B_c = np.array([[0],
                [1/m]])
C_true = np.eye(2)

C = [
    np.array([[1, 0], [0, 1]]),
    np.array([[1, 0], [0, 0]]),
    np.array([[0, 0], [0, 1]])
]
# Process and measurement noise
Q = np.array([[1e-4, 0], 
              [0, 1e-4]])  # Process noise covariance
R = np.array([[0.05, 0], 
              [0, 0.05]])  # Measurement noise covariance

# Discrete time system for kalman filter
dt = 0.1  # Time step
A_d = expm(A_c * dt)
B_d = np.linalg.solve(A_c, (A_d - np.eye(2))) @ B_c

# Initial conditions
x0 = np.array([1.0, 0.0])  # Initial displacement and velocity


Now, simulate the true system. 

In [127]:
# Simulate the true system
np.random.seed(42)
n_steps = 100 
true_states = []
measurements = []
for _ in range(n_steps):
    # Simulate true dynamics
    x_true = A_d @ x0 + np.random.multivariate_normal([0, 0], Q).T
    true_states.append(x_true)

    # Simulate noisy measurements
    z = C_true @ x_true + np.random.multivariate_normal([0, 0], R).T
    measurements.append(z)

    # Update for the next time step
    x0 = x_true

true_states = np.array(true_states)
measurements = np.array(measurements)

Then, use the  Kalman filter to generate state estimates from simulated measurments. Make sure to use the 3 sigma bound to set up the estimation range. This block runs the Kalman Filter for every subgroup of C matrices. 

In [ ]:
nums = np.zeros(shape=(n_steps, 4))
c1_data = pd.DataFrame(nums, columns=['position', 'velocity', 'position uncertainty', 'velocity uncertainty'])
c2_data = pd.DataFrame(nums, columns=['position', 'velocity', 'position uncertainty', 'velocity uncertainty'])
c3_data = pd.DataFrame(nums, columns=['position', 'velocity', 'position uncertainty', 'velocity uncertainty'])

for i in range(len(C)):
    system = (A_c, B_c, np.eye(2), 0)
    A_d, B_d, _, _, _ = cont2discrete(system, dt)
    Q = np.eye(2) * 1e-4
    R = np.eye(2) * 0.05

    # Kalman Filter setup
    kf = KalmanFilter(dim_x=2, dim_z=2)
    kf.F = A_d  # Discretized A matrix
    kf.H = C[i]  # Measurement matrix C
    kf.Q = Q  # Process noise covariance Q
    kf.R = R  # Measurement noise covariance R
    kf.P = np.eye(2) * 500  # Initial state covariance P
    kf.x = x0  # Initial state estimate

    state_at_this_C = []
    uncertainty_at_this_C = []

    for z in measurements:
        kf.predict()
        kf.update(z)
        state_at_this_C.append(kf.x.copy())
        uncertainty_at_this_C.append(3 * np.sqrt(np.diag(kf.P))) # make sure to multiply by 3 for 3 sigma!!!


    position  velocity  position uncertainty  velocity uncertainty
0        0.0       0.0                   0.0                   0.0
1        0.0       0.0                   0.0                   0.0
2        0.0       0.0                   0.0                   0.0
3        0.0       0.0                   0.0                   0.0
4        0.0       0.0                   0.0                   0.0
..       ...       ...                   ...                   ...
95       0.0       0.0                   0.0                   0.0
96       0.0       0.0                   0.0                   0.0
97       0.0       0.0                   0.0                   0.0
98       0.0       0.0                   0.0                   0.0
99       0.0       0.0                   0.0                   0.0

[100 rows x 4 columns]


Finally, plot the results. 

In [129]:
# Plot results
